## Prediction of the relative location of CT slices on axial axis

### Support code section

In [1]:
from ct_support_code import *

### Question 1

In [2]:
## loading data
data = np.load('ct_data.npz')

## X's
X_train = data['X_train']; X_val = data['X_val']; X_test = data['X_test']

## y's
y_train = data['y_train']; y_val = data['y_val']; y_test = data['y_test']

In [3]:
## calculating requested means and std errors on those means 

## training data

print("Mean of y_train:",np.mean(y_train))
print("Mean of y_train (for 5785 entries):", np.mean(y_train[:5785]))
print("Standard error for y_train (for 5785 entries):",np.std(y_train[:5785], ddof=1)/np.sqrt(len(y_train[:5785])))

## validation data

print("Mean of y_val (for 5785 entries):",np.mean(y_val))
print("Standard error for y_val (for 5785 entries):",np.std(y_val, ddof=1)/np.sqrt(len(y_val)))

Mean of y_train: -9.13868774539957e-15
Mean of y_train (for 5785 entries): -0.44247687859693674
Standard error for y_train (for 5785 entries): 0.011927303389170828
Mean of y_val (for 5785 entries): -0.2160085093241599
Standard error for y_val (for 5785 entries): 0.01290449880016868


In [4]:
## removing constant and duplicate features

## constants

rm_idx0=[]    

for i in range(len(X_train[1])):
    col=X_train[:,i]
    if all(col[0]==col):
        rm_idx0.append(i)
print("Removed constant columns (as in the original array) indices are:", rm_idx0) 


X_train = np.delete(X_train,rm_idx0,axis=1)
X_val = np.delete(X_val, rm_idx0, axis=1)
X_test = np.delete(X_test, rm_idx0, axis=1)


## duplicates

indices= np.unique(X_train,return_index=True,axis=1)[1]

indices=np.sort(indices)
rm_idx=list(set(range(indices[0],indices[-1]+1))-set(indices))

X_train = np.delete(X_train,rm_idx,axis=1)
X_val = np.delete(X_val, rm_idx, axis=1)
X_test = np.delete(X_test, rm_idx, axis=1)

print("Removed duplicate column (as in the original array) indices are:",rm_idx)


Removed constant columns (as in the original array) indices are: [59, 69, 179, 189, 351]
Removed duplicate column (as in the original array) indices are: [354, 195, 76, 77, 185, 283]


### Question 2

In [5]:
def fit_linreg(X, yy, alpha):
    k=len(X[1])                                 ## getting number of input features
    yy = np.concatenate((yy, np.zeros(k)))      ## adding 0_k to the y_train array
    z_k = np.sqrt(alpha) * np.eye(k)
    X = np.vstack((X,z_k))  
    
    b = np.concatenate((np.ones(len(X)-k), np.zeros(k)))[:,None]

    X = np.insert(X,[0],b,axis=1)

    w_fit=np.linalg.lstsq(X, yy, rcond=None)[0]
    
    
    return w_fit[1:], w_fit[0]

In [6]:
alpha=30

## least squares weights & bias
ww0, bb0 = fit_linreg(X_train, y_train, alpha)

## gradient method weights & bias
ww1,bb1 = fit_linreg_gradopt(X_train, y_train, 30)

In [7]:
## defining rmse

def rmse(pred,yy):
    return np.sqrt(np.mean((pred-yy)**2))

In [8]:
## for least squares method

pred1_train = np.dot(X_train,ww0)+bb0
pred1_val = np.dot(X_val,ww0)+bb0
print("RMSE for training set (using least squares method):",rmse(pred1_train, y_train))
print("RMSE for validation set (using least squares method):",rmse(pred1_val, y_val))

RMSE for training set (using least squares method): 0.3567565397204054
RMSE for validation set (using least squares method): 0.42305219683946976


In [9]:
## for gradient method

pred2_train = np.dot(X_train,ww1)+bb1
pred2_val = np.dot(X_val,ww1)+bb1
print("RMSE for training set (using gradient method):",rmse(pred2_train, y_train))
print("RMSE for validation set (using gradient method):",rmse(pred2_val, y_val))

RMSE for training set (using gradient method): 0.35675597702036305
RMSE for validation set (using gradient method): 0.4230550153899789


### Question 3

In [10]:
## modified function

def fit_logreg_gradopt(X, yy, alpha):
    """
    fit a regularized linear regression model with gradient opt

         ww, bb = fit_linreg_gradopt(X, yy, alpha)

     Find weights and bias by using a gradient-based optimizer
     (minimize_list) to improve the regularized least squares cost:

       np.sum(((np.dot(X,ww) + bb) - yy)**2) + alpha*np.dot(ww,ww)

     Inputs:
             X N,D design matrix of input features
            yy N,  real-valued targets
         alpha     scalar regularization constant

     Outputs:
            ww D,  fitted weights
            bb     scalar fitted bias
    """
    D = X.shape[1]
    args = (X, yy, alpha)
    init = (np.zeros(D), np.array(0))
    ww, bb = minimize_list(logreg_cost, init, args)
    return ww, bb



K = 20 # number of thresholded classification problems to fit
mx = np.max(y_train); mn = np.min(y_train); hh = (mx-mn)/(K+1)
thresholds = np.linspace(mn+hh, mx-hh, num=K, endpoint=True)

w_fit2= np.array([[0.0]* (len(X_train[1])+1)] * K)
for kk in range(K):
    labels = y_train > thresholds[kk]
    ww2, bb2 = fit_logreg_gradopt(X_train, labels, alpha=30)
    w_fit2[kk,0] = bb2
    w_fit2[kk,1:]=ww2

In [11]:
bb2_hat = w_fit2[:,0]
ww2_hat = w_fit2[:,1:]

## defining sigmoid

def sigmoid(a):
    return 1 / (1+np.exp(-a))



X_train_new = sigmoid(np.dot(X_train, np.transpose(ww2_hat))+bb2_hat) 
X_val_new = sigmoid(np.dot(X_val, np.transpose(ww2_hat))+bb2_hat)


nn_ww, nn_bb = fit_linreg(X_train_new, y_train, alpha=30)

In [12]:
pred3_train = np.dot(X_train_new, nn_ww) + nn_bb
pred3_val = np.dot(X_val_new, nn_ww) + nn_bb

print("RMSE for training set:",rmse(pred3_train, y_train))
print("RMSE for validation set:",rmse(pred3_val, y_val))

RMSE for training set: 0.1544115042984819
RMSE for validation set: 0.2542477297888156


### Question 4

In [13]:
np.random.seed(42) ## for random weight initialization


def fit_nn_gradopt(X, yy, K, alpha, w_random = True):
    """
    fit a regularized linear regression model with gradient opt

         ww, bb = fit_linreg_gradopt(X, yy, alpha)

     Find weights and bias by using a gradient-based optimizer
     (minimize_list) to improve the regularized least squares cost:

       np.sum(((np.dot(X,ww) + bb) - yy)**2) + alpha*np.dot(ww,ww)

     Inputs:
             X N,D design matrix of input features
            yy N,  real-valued targets
         alpha     scalar regularization constant
         w_random  controls random initialisation of weights

     Outputs:
            ww D,  fitted weights
            bb     scalar fitted bias
    """
    args = (X, yy, alpha)
    
    if w_random:
        
        D = len(X_train[1])
       
    
        # generate random initialisation
        ww = 0.1 * np.random.randn(K)/np.sqrt(K)
        V = 0.1 * np.random.randn(K,D)/np.sqrt(D)
        bk = np.zeros(K)
        bb= 0
        init = (ww, bb, V, bk)
        ww, bb, V, bk = minimize_list(nn_cost, init, args)
        return (ww, bb, V, bk)
    
    else:
        
        init = (nn_ww,nn_bb,ww2_hat, bb2_hat)            ## Initialization from the results we obtained 
        ww, bb, V, bk = minimize_list(nn_cost, init, args)
        return (ww, bb, V, bk)

In [14]:
params = fit_nn_gradopt(X_train, y_train,K=20, alpha=30)

pred_train_nn= nn_cost(params, X_train, yy=None, alpha=30)
pred_val_nn= nn_cost(params, X_val, yy=None, alpha=30)

In [15]:
print("Training set RMSE for NN (with random initialization):",rmse(pred_train_nn, y_train))
print("Validation set RMSE for NN (with random initialization):",rmse(pred_val_nn, y_val))

Training set RMSE for NN (with random initialization): 0.14023263982522416
Validation set RMSE for NN (with random initialization): 0.27047491191944456


In [16]:
params2 = fit_nn_gradopt(X_train, y_train,K=20, alpha=30, w_random = False)

pred1_train_nn= nn_cost(params2, X_train, yy=None, alpha=30)
pred1_val_nn= nn_cost(params2, X_val, yy=None, alpha=30)

In [17]:
print("Training set RMSE for NN (with initialization from q3):",rmse(pred1_train_nn, y_train))
print("Validation set RMSE for NN (with initialization from q3):",rmse(pred1_val_nn, y_val))

Training set RMSE for NN (with initialization from q3): 0.13963009839601498
Validation set RMSE for NN (with initialization from q3): 0.26857721499074866


### Question 5

In [18]:
def train_nn_reg(X_train, X_val, yy, y_val, train_alpha):
    
    param = fit_nn_gradopt(X_train, yy, K=20, alpha= train_alpha)
    
    pred_val = nn_cost(param, X_val, yy=None, alpha= train_alpha)

    return (rmse(pred_val,y_val), param)
    
    
alpha= np.arange(0,50,0.02)

indicies = np.random.choice(len(alpha),3) 
obs_alpha = np.array(alpha[indicies])
test_alpha = np.delete(alpha,indicies)

obs_alpha_val = np.array([])

for alpha in obs_alpha: 
    val_rmse = train_nn_reg(X_train, X_val, y_train, y_val, alpha)[0]
    obs_alpha_val = np.append(obs_alpha_val, val_rmse )
    

In [19]:
import scipy.stats

log_base_rmse = np.log(rmse(pred_val_nn, y_val))    ## Validation rmse from question 4a

y = np.array(log_base_rmse - np.log(obs_alpha_val))

post_mean, post_cov  = gp_post_par(test_alpha, obs_alpha, y)
 
post_std = np.sqrt(np.diag(post_cov))         ## Standard deviation

def phi(post_mean, post_std, y):
    return scipy.stats.norm.cdf((post_mean - max(y))/post_std)

best_alpha = 0.0
best_alpha_rmse = 9999.0
best_params = set()
for _ in range(5):
    prob_max = phi(post_mean, post_std, y)
    idx = np.argmax(prob_max)
    
    
    alpha_val_rmse, params = train_nn_reg(X_train, X_val, y_train, y_val, test_alpha[idx])
    
    if  alpha_val_rmse < best_alpha_rmse:
        best_alpha = test_alpha[idx]
        best_alpha_rmse = alpha_val_rmse
        best_params = params
    
    print("Probability of improvement for Alpha(={0}) is {1} and Validation RMSE is: {2}".format( 
                                test_alpha[idx], prob_max[idx], alpha_val_rmse))
    
    obs_alpha_val = np.append(obs_alpha_val, alpha_val_rmse)
    
    obs_alpha = np.append(obs_alpha, test_alpha[idx])
    test_alpha = np.delete(test_alpha,idx)

    y = np.array(log_base_rmse - np.log(obs_alpha_val))
    post_mean, post_cov  = gp_post_par(test_alpha, obs_alpha, y)
    post_std = np.sqrt(np.diag(post_cov))

Probability of improvement for Alpha(=0.0) is 0.5827825928435952 and Validation RMSE is: 0.27360064710085197
Probability of improvement for Alpha(=11.200000000000001) is 0.5406420072971778 and Validation RMSE is: 0.2520158993589383
Probability of improvement for Alpha(=11.58) is 0.37340378241418304 and Validation RMSE is: 0.25856636671937655
Probability of improvement for Alpha(=8.82) is 0.32007921055127664 and Validation RMSE is: 0.25322312424903504
Probability of improvement for Alpha(=8.84) is 0.34952151149666943 and Validation RMSE is: 0.26230654140428356


In [20]:
## Traning on best alpha to get test error
pred_test = nn_cost(best_params, X_test, yy=None, alpha=best_alpha)   ## Prediction for test set
test_error = rmse(pred_test, y_test)

print("The Best value for alpha is {0}".format(best_alpha))
print("The Validation error is {0}".format(best_alpha_rmse))
print("The Test error is {0}".format(test_error))

The Best value for alpha is 11.200000000000001
The Validation error is 0.2520158993589383
The Test error is 0.27222277241918336


In [21]:
train_nn_reg(X_train, X_val, y_train, y_val, best_alpha)[0]

0.2589297557830086

In [22]:
train_nn_reg(X_train, X_val, y_train, y_val, best_alpha)[0]

0.2626651816225229

In [25]:
def mod_train_nn_reg(X_train, X_val, yy, y_val, train_alpha, KK):
    
    param = fit_nn_gradopt(X_train, yy, K=KK, alpha= train_alpha)
    
    pred_val = nn_cost(param, X_val, yy=None, alpha= train_alpha)

    return (rmse(pred_val,y_val), param)

In [34]:
## vector of different hidden neurons to try 

KK = np.array([10,20,30,40,50,60,70,80])

## we iteratively apply what we did in q5 

for kk in KK:
    
    alpha= np.arange(0,50,0.02)

    indicies = np.random.choice(len(alpha),3) 
    obs_alpha = np.array(alpha[indicies])
    test_alpha = np.delete(alpha,indicies)

    obs_alpha_val = np.array([])

    for alpha in obs_alpha: 
        val_rmse = mod_train_nn_reg(X_train, X_val, y_train, y_val, alpha,kk)[0]
        obs_alpha_val = np.append(obs_alpha_val, val_rmse )
    
    y = np.array(log_base_rmse - np.log(obs_alpha_val))
    post_mean, post_cov  = gp_post_par(test_alpha, obs_alpha, y)
    post_std = np.sqrt(np.diag(post_cov))         ## Standard deviation
    best_alpha = 0.0
    best_alpha_rmse = 9999.0
    best_params = set()
    for _ in range(5):
        prob_max = phi(post_mean, post_std, y)
        idx = np.argmax(prob_max)
    
    
        alpha_val_rmse, params = mod_train_nn_reg(X_train, X_val, y_train, y_val, test_alpha[idx],kk)
    
        if  alpha_val_rmse < best_alpha_rmse:
            best_alpha = test_alpha[idx]
            best_alpha_rmse = alpha_val_rmse
            best_params = params
    
        obs_alpha_val = np.append(obs_alpha_val, alpha_val_rmse)
    
        obs_alpha = np.append(obs_alpha, test_alpha[idx])
        test_alpha = np.delete(test_alpha,idx)

        y = np.array(log_base_rmse - np.log(obs_alpha_val))
        post_mean, post_cov  = gp_post_par(test_alpha, obs_alpha, y)
        post_std = np.sqrt(np.diag(post_cov))
        
    print("Number of hidden units{0}".format(kk))
    print("Best alpha{0}".format(best_alpha))
    print("Validation set RMSE{0}".format(best_alpha_rmse))
    

Number of hidden units10
Best alpha14.36
Validation set RMSE0.258420627930365
Number of hidden units10
Best alpha14.36
Validation set RMSE0.258420627930365
Number of hidden units10
Best alpha14.36
Validation set RMSE0.258420627930365
Number of hidden units10
Best alpha14.36
Validation set RMSE0.258420627930365
Number of hidden units10
Best alpha14.36
Validation set RMSE0.258420627930365
Number of hidden units20
Best alpha32.160000000000004
Validation set RMSE0.27748351430027884
Number of hidden units20
Best alpha0.0
Validation set RMSE0.260600482202473
Number of hidden units20
Best alpha0.02
Validation set RMSE0.2540269207381081
Number of hidden units20
Best alpha3.2
Validation set RMSE0.22684314477771944
Number of hidden units20
Best alpha3.2
Validation set RMSE0.22684314477771944
Number of hidden units30
Best alpha4.6000000000000005
Validation set RMSE0.24898237963583714
Number of hidden units30
Best alpha4.6000000000000005
Validation set RMSE0.24898237963583714
Number of hidden unit

KeyboardInterrupt: 